Question 1

In [ ]:
import json

def count_chickens(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    chicken_count = sum(1 for shape in data['shapes'] if shape['label'] == 'chicken')
    return chicken_count

# Example usage
json_path = '496.json'
print("Number of chickens:", count_chickens(json_path))


Number of chickens: 125


Question 2

In [9]:
import mediapipe as mp
import cv2
import os
import pandas as pd

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils


image_folder = "activity_dataset"

# Automatically get image paths per activity
image_info = {}
for activity_name in os.listdir(image_folder):
    activity_path = os.path.join(image_folder, activity_name)
    if os.path.isdir(activity_path):
        # Get all images in this activity folder
        image_list = [os.path.join(activity_path, f) 
                      for f in os.listdir(activity_path) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        image_info[activity_name] = image_list

# Output data
output_data = []

for activity, image_paths in image_info.items():
    for image_path in image_paths:
        image_name = os.path.basename(image_path)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load: {image_name}")
            continue

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = pose.process(image_rgb)

        # Collect keypoints
        keypoints = []
        if result.pose_landmarks:
            for landmark in result.pose_landmarks.landmark:
                keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
        else:
            keypoints = [None] * (33 * 4)  # If pose not detected

        # Append to output
        output_data.append({
            "file_name": image_name,
            "activity": activity,
            "keypoints": keypoints
        })

# Convert to DataFrame
df = pd.DataFrame(output_data)

# Save to CSV or JSON
# df.to_csv("human_keypoints_with_activity.csv", index=False)
df.to_json("human_keypoints_with_activity.json", orient='records', indent=2)



Keypoint Visualization Check: Plotting Keypoint from JSON on Original Image.

In [2]:
import json
import mediapipe as mp

# Load JSON file
with open("human_keypoints_with_activity.json", "r") as f:
    data = json.load(f)

# Initialize MediaPipe drawing utilities
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose


In [3]:
import cv2
import json

# Load JSON
with open("human_keypoints_with_activity.json", "r") as f:
    data = json.load(f)

# Pose connections (MediaPipe's POSE_CONNECTIONS)
POSE_CONNECTIONS = mp.solutions.pose.POSE_CONNECTIONS

for entry in data:
    activity = entry['activity']
    file_name = entry['file_name']
    keypoints = entry['keypoints']

    image_file = f"activity_dataset/{activity}/{file_name}"
    image = cv2.imread(image_file)
    if image is None:
        continue

    h, w, _ = image.shape

    # Convert keypoints to pixel coordinates
    points = []
    for i in range(33):
        x, y, z, v = keypoints[i*4:(i+1)*4]
        points.append((int(x*w), int(y*h)))

    # Draw keypoints
    for point in points:
        cv2.circle(image, point, 5, (0, 255, 0), -1)

    # Draw connections
    for connection in POSE_CONNECTIONS:
        start_idx, end_idx = connection
        pt1, pt2 = points[start_idx], points[end_idx]
        cv2.line(image, pt1, pt2, (0, 0, 255), 2)

    cv2.imshow(file_name, image)
    cv2.waitKey(2000)
    cv2.destroyWindow(file_name)
